In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Input, Dense, GlobalAveragePooling2D, merge, Dropout, Convolution2D, Lambda
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a 10 legtöbb képpel rendelkező alkotót kiválogatjuk
authors_to_select = author_stat[-100:]
authors_to_select

[('358474690052f63f9107c823b7a61686', 165),
 ('342d7068656b00878b23e1031a3ddf50', 165),
 ('db85b84c1c01fa143b9294f00f7acbeb', 167),
 ('30134467c49a3350f74b4311506f8386', 171),
 ('795985c5c0e7c704a354a487bdffeef5', 171),
 ('ed9cf03b71a8a71d41ecb4e8406a0877', 172),
 ('a087f33967e659c63924aa07c89b948f', 172),
 ('3a320926ef941d53716127548523450c', 172),
 ('24ed88087b0ffeb763e89a197a5cde0d', 172),
 ('a7b6670e2d23f415998849a8fdedae18', 174),
 ('8b1800319323ab9b0ce2e0b11b3f4772', 175),
 ('79f6b47483600c140c91e901f746a368', 176),
 ('3b4ee6d6140803ebef405983f23ffcc1', 176),
 ('56221362cba30b7e0c96111cc38c5fcd', 176),
 ('83518d02afc5754dd9e79985f56c3355', 178),
 ('9c948f31fc16aca442cc91e7fe05fb95', 179),
 ('b31c8ec49c09506f17ec3c0c04fce567', 179),
 ('062244b785e58518f2f189329633c59e', 181),
 ('63e73f693e541f53d98ed6c93d03ef40', 185),
 ('39c465a1d078121873a31807ce9dc2ed', 185),
 ('ea2beb11e25ec317263983dbdd6de81e', 186),
 ('1d6d112c140cb6d1d763a7ca73f2eec0', 187),
 ('74a1dad53214d04514269a188f166

In [3]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, "train_sample")

5 images in the result


In [4]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[3 1 2 0 4]
[[ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]]


In [5]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [6]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

In [7]:
im_input = Input(shape=(256,256,3))

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=im_input)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót

In [8]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
desired_layers = [28, 44, 60, 70, 92, 114, 136, 158, 172, 194]
style_layers = [None]*len(desired_layers)

for i in range(len(desired_layers)):
    style_layers[i] = base_model.layers[desired_layers[i]].output
    style_layers[i] = GlobalAveragePooling2D()(style_layers[i])
    style_layers[i] = Dense(base_model.layers[desired_layers[i]].output_shape[3], activation='relu')(style_layers[i])

In [9]:
# egymás mellé tesszük a különböző szintű feature-öket
style_output = merge(style_layers, mode='concat', name='style_output')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(2048, activation='relu')(style_output)
ff = Dropout(0.5)(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [10]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=1, validation_split=0.2, callbacks=[history])

Train on 4 samples, validate on 1 samples
Epoch 1/1
4/4 [==============================] - 3s - loss: 1.5287 - acc: 0.5000 - val_loss: 16.1181 - val_acc: 0.0000e+00


In [12]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [13]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [14]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=1, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 4 samples, validate on 1 samples
Epoch 1/1
4/4 [==============================] - 4s - loss: 0.0898 - acc: 1.0000 - val_loss: 12.5789 - val_acc: 0.0000e+00
Tanítás vége.


In [15]:
predicted_values = model.predict(np.asarray(train_images))

In [16]:
style_out = Model(input=base_model.input, output=style_output)
styles = style_out.predict(np.asarray(train_images))
style_dict = {}
for i in range(len(labels)):
    if(labels[i] in style_dict):
        style_dict[labels[i]] = (style_dict[labels[i]] + styles[i]) * 0.5
    else:
        style_dict[labels[i]] = styles[i]
    
print(style_dict)

{'3cc9a44380296d93e68b71a27643c25f': array([ 0.        ,  0.        ,  0.71451682, ...,  0.        ,
        0.102036  ,  0.20697585], dtype=float32), 'e1587900e782de448f604b37cde0fdfd': array([ 0.        ,  0.        ,  0.30844185, ...,  0.05654748,
        0.        ,  0.38040966], dtype=float32), '8e441c5899bf3d2f3b2c493e62fb92bf': array([ 0.56115144,  0.        ,  1.09738946, ...,  0.021096  ,
        0.03279456,  0.65711933], dtype=float32), 'cc47068929413a16aa707faefbdf4b70': array([ 1.11500239,  0.        ,  0.37770706, ...,  0.        ,
        0.        ,  0.16324107], dtype=float32), 'c56bcab4b317984013ebef5d3c4b5906': array([ 0.        ,  0.        ,  0.84439641, ...,  0.27395222,
        0.        ,  0.15159859], dtype=float32)}


In [17]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

[3, 1, 2, 0, 1]

In [18]:
encoder.inverse_transform(festok)

array(['cc47068929413a16aa707faefbdf4b70',
       '8e441c5899bf3d2f3b2c493e62fb92bf',
       'c56bcab4b317984013ebef5d3c4b5906',
       '3cc9a44380296d93e68b71a27643c25f',
       '8e441c5899bf3d2f3b2c493e62fb92bf'], 
      dtype='<U32')

In [19]:
ones = np.ones((256,256,3))

ones_input = Input(shape =(256,256,3))
synth_layer = Convolution2D(3, 256, 256, init='glorot_uniform', weights=None, border_mode='same', subsample=(1, 1), bias=True)(ones_input)
#style_loss = style_out(synth_layer)

In [20]:
'''Deep Dreaming in Keras.

Run the script with:
```
python deep_dream.py path_to_your_base_image.jpg prefix_for_results
```
e.g.:
```
python deep_dream.py img/mypic.jpg results/dream
```

It is preferable to run this script on GPU, for speed.
If running on CPU, prefer the TensorFlow backend (much faster).

Example results: http://i.imgur.com/FX6ROg9.jpg
'''
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from scipy.misc import imsave
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg16
from keras import backend as K
from keras.layers import Input

In [ ]:
# dimensions of the generated picture.
img_width = 256
img_height = 256

# path to the model weights file.
weights_path = 'vgg16_weights.h5'

# some settings we found interesting
saved_settings = {
    'bad_trip': {'features': {'block4_conv1': 0.05,
                              'block4_conv2': 0.01,
                              'block4_conv3': 0.01},
                 'continuity': 0.1,
                 'dream_l2': 0.8,
                 'jitter': 5},
    'dreamy': {'features': {'convolution2d_2': 0.05,
                            'convolution2d_3': 0.05},
               'continuity': 0.1,
               'dream_l2': 0.02,
               'jitter': 0},
}

In [70]:
def preprocess_image():
    img = np.random.random((256,256,3))
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image
def deprocess_image(x):
    if K.image_dim_ordering() == 'th':
        x = x.reshape((3, img_width, img_height))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_width, img_height, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

if K.image_dim_ordering() == 'th':
    img_size = (3, img_width, img_height)
else:
    img_size = (img_width, img_height, 3)
# this will contain our generated image
dream = Input(batch_shape=(1,) + img_size)

In [71]:
# build the VGG16 network with our placeholder
# the model will be loaded with pre-trained ImageNet weights
model2 = vgg16.VGG16(input_tensor=dream, weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


In [86]:
# the settings we will use in this experiment
settings = saved_settings['dreamy']


# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [87]:
# continuity loss util function
def continuity_loss(x):
    assert K.ndim(x) == 4
    if K.image_dim_ordering() == 'th':
        a = K.square(x[:, :, :img_width - 1, :img_height - 1] -
                     x[:, :, 1:, :img_height - 1])
        b = K.square(x[:, :, :img_width - 1, :img_height - 1] -
                     x[:, :, :img_width - 1, 1:])
    else:
        a = K.square(x[:, :img_width - 1, :img_height-1, :] -
                     x[:, 1:, :img_height - 1, :])
        b = K.square(x[:, :img_width - 1, :img_height-1, :] -
                     x[:, :img_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [88]:
# define the loss
loss = K.variable(0.)
for layer_name in settings['features']:
    # add the L2 norm of the features of a layer to the loss
    assert layer_name in layer_dict.keys(), 'Layer ' + layer_name + ' not found in model.'
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    print(layer_dict[layer_name])
    x = x[0,:,:,:]
    print(x)
    shape = layer_dict[layer_name].output_shape
    print(shape)
    # we avoid border artifacts by only involving non-border pixels in the loss
    loss -= coeff * K.sum(K.square(x[2: shape[1] - 2, 2: shape[2] - 2, :])) / np.prod(shape[1:])

# add continuity loss (gives image local coherence, can result in an artful blur)
loss += settings['continuity'] * continuity_loss(dream) / np.prod(img_size)
# add image L2 norm to loss (prevents pixels from taking very high values, makes image darker)
loss += settings['dream_l2'] * K.sum(K.square(dream)) / np.prod(img_size)

# feel free to further modify the loss as you see fit, to achieve new effects...

Tensor("Squeeze_121:0", shape=(125, 125, 32), dtype=float32)
(None, 125, 125, 32)
Tensor("Squeeze_122:0", shape=(125, 125, 64), dtype=float32)
(None, 125, 125, 64)


In [89]:
# compute the gradients of the dream wrt the loss
grads = K.gradients(loss, dream)
print(loss)
outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([dream], outputs)
def eval_loss_and_grads(x):
    x = x.reshape((1,) + img_size)
    print(x)
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

Tensor("add_331:0", shape=(), dtype=float32)


In [90]:
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the loss
x = preprocess_image()
for i in range(5):
    print('Start of iteration', i)
    start_time = time.time()

    # add a random jitter to the initial image. This will be reverted at decoding time
    random_jitter = (settings['jitter'] * 2) * (np.random.random(img_size) - 0.5)
    x += random_jitter

    # run L-BFGS for 7 steps
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=7)
    print('Current loss value:', min_val)
    # decode the dream and save it
    x = x.reshape(img_size)
    x -= random_jitter
    img = deprocess_image(np.copy(x))
    fname = 'resuat_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
[[[[-103.66170519 -116.44717959 -123.52004745]
   [-103.51080197 -116.44416334 -122.79904106]
   [-103.55824657 -115.78824907 -122.7704693 ]
   ..., 
   [-103.49367728 -116.67054468 -123.47167165]
   [-103.61399327 -115.87243195 -123.13025461]
   [-102.9782557  -115.96227704 -122.77619619]]

  [[-103.01159095 -116.39104603 -123.37518834]
   [-103.82858565 -115.84573768 -123.46859101]
   [-103.28007695 -115.82469597 -123.55114392]
   ..., 
   [-103.28695538 -115.80719137 -123.25847049]
   [-103.45643999 -115.89458082 -123.04602775]
   [-103.20019628 -116.14888766 -123.30115994]]

  [[-103.24851191 -116.56837697 -123.54113148]
   [-103.3799732  -116.0481408  -122.94516678]
   [-103.799905   -116.50604127 -123.65660377]
   ..., 
   [-103.30758138 -115.98334972 -123.16692325]
   [-102.98577493 -115.96931958 -123.62185359]
   [-103.10433609 -116.67282719 -123.34295628]]

  ..., 
  [[-103.15535476 -116.71088936 -123.23716906]
   [-103.4250752  -116.12548976 -123.02520804

InvalidArgumentError: You must feed a value for placeholder tensor 'input_1' with dtype float
	 [[Node: input_1 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'input_1', defined at:
  File "/home/kohlmann/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kohlmann/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kohlmann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/kohlmann/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kohlmann/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/kohlmann/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(Z